Intro - Home Work 

#1 Answer : version.build_hash = "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
#documents

In [1]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [2]:
index_name = "course-qna"

#2 Answer : Which function do you use for adding your data to elastic? - index

In [3]:
query = "How do I execute a command in a running docker container?"

In [4]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }



In [5]:
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 390, 'relation': 'eq'}, 'max_score': 75.54128, 'hits': [{'_index': 'course-qna', '_id': '8F1pSpABcBxGRfciTf0y', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|------

In [6]:
response['hits']['max_score']

75.54128

In [7]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }


In [8]:
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-qna', '_id': '4l1pSpABcBxGRfcivP8r', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-qna', '_id': 'AV5pSpABcBxGRfciwgA5', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file o

In [9]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [10]:
print(result_docs)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'm

In [34]:
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [ ]:
#  context_template = """
#  Q: {question}
#  A: {text}
#  """.strip()

In [11]:
context = ""
for doc in result_docs:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [12]:
question = "How do I execute a command in a running docker container?"

In [13]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [14]:
prompt = prompt_template.format(question=query, context=context).strip()

In [15]:
len(prompt)

1462

In [16]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [17]:
import tiktoken

In [21]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
num_tokens = len(encoding.encode(prompt))

In [23]:
print(num_tokens)

322


In [24]:
import os
import openai
from openai import OpenAI

In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
client = OpenAI()

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages = [{"role" : "user" , "content" : prompt}]
    )
    return response.choices[0].message.content

In [28]:
answer = llm(prompt)

In [29]:
print(answer)

To execute a command in a running Docker container, follow these steps:

1. Identify the container ID of the running Docker container using the `docker ps` command:
   ```sh
   docker ps
   ```
   This command will list all running containers and their corresponding container IDs.

2. Use the `docker exec` command to execute a command within the identified container. For example, to start a bash session in the specific container:
   ```sh
   docker exec -it <container-id> bash
   ```
Replace `<container-id>` with the actual container ID obtained from the `docker ps` output.

This will open an interactive bash session in the running container, allowing you to execute further commands as needed.
